# T0. PREPARE POINT CLOUDS

Import and prepare CVPR point clouds for t1-t10.
To run these scripts, create a python 3.10 environment & install geomapi (numpy, opend3d, ifcopenshell, trimesh, ...)

## LIBRARIES

In [2]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import time
import json
from scipy.spatial.transform import Rotation   
import copy
from datetime import datetime

import geomapi
from geomapi.nodes import *
from geomapi import utils as ut
from geomapi.utils import geometryutils as gmu
from geomapi.tools import progresstools as pt

import geomapi.tools as tl

import context
import utils as utl
import utils.t0_utils as t0


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

## INPUTS

In [5]:
start_time = time.time()

In [63]:
path=Path(os.getcwd()).parents[2]/'data'
input_folder=path / 't8'/'test'
class_file=path/'_classes.json'
output_folder= path/'results'/'test_2'
output_folder.mkdir(parents=True, exist_ok=True)
graphPath=str(path/f'objectGraph.ttl')

distance_threshold=0.1
resolution=0.03

In [9]:
print(path)

c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data


import classes

In [64]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'beams', 'id': 5, 'temp_id': 6, 'color': '#79c314'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


import objects

In [67]:
files=utl.get_list_of_files(input_folder, '.json')
objects_dict = {}  
objectNodes=[]
for file in files[1:2]:
    
    # Load the JSON data into Python objects
    t0.parse_json(file,objects_dict)

# Print counts
print(f'len columns: {sum(1 for item in objects_dict.values() if item["type"] == "columns")}')
print(f'len doors: {sum(1 for item in objects_dict.values() if item["type"] == "doors")}')
print(f'len walls: {sum(1 for item in objects_dict.values() if item["type"] == "walls")}')

#convert json to object nodes
objectNodes.extend(t0.create_object_nodes(objects_dict,class_dict))

print(f'len objectNodes: {len(objectNodes)}')

Data read from file: 08_ShortOffice_01_F2_doors
len columns: 0
len doors: 31
len walls: 0
len objectNodes: 31


In [71]:
joined_geometries2=gmu.join_geometries([d.resource.paint_uniform_color([0.9,0.9,0.9]) for d in objectNodes if d.class_id==4])

o3d.visualization.draw_geometries([joined_geometries2])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [52]:
# bad_geometries=gmu.join_geometries([d.resource.paint_uniform_color([1,0,0]) for d in objectNodes if (d.class_id==3) and  
#                                                                                                         ((d.width < 0.2 or d.depth < 0.2) or
#                                                                                                         ((d.width / d.depth)> 2.0) or ((d.depth / d.width)> 2))  ])
# good_geometries=gmu.join_geometries([d.resource.paint_uniform_color([0,1,0]) for d in objectNodes if (d.class_id==3) and  
#                                                                                                         ((d.width > 0.2 and d.depth > 0.2) and
#                                                                                                         ((d.width / d.depth)< 2) and ((d.depth / d.width)< 2))  ])
# joined_geometries2=gmu.join_geometries([d.resource.paint_uniform_color([0.9,0.9,0.9]) for d in objectNodes if d.class_id==2])

# # lines=gmu.join_geometries([n.line.paint_uniform_color(n.color) for n in objectNodes if n.derivedFrom==pcdNode.name 
# #                            if getattr(n,'line',None) is not None])

# o3d.visualization.draw_geometries([bad_geometries,good_geometries,joined_geometries2])

Import PCD

In [69]:
input_folder_t4=path/'t4'/'test' 

files=utl.get_list_of_files(input_folder_t4, '.laz')

#iterate over files
for f_pcd in files[1:2]:     
    
    print(f'processing {ut.get_filename(f_pcd)}...') 
    las = laspy.read(f_pcd) #if 'las' not in globals() else las
    pcd=gmu.las_to_pcd(las) #if 'pcd' not in globals() else pcd # this is the slowest step
        
    wallNodes=[n for n in objectNodes if n.class_id==2]    

    #create an identity point cloud of all the objectNodes
    identityPcd,identityArray=gmu.create_identity_point_cloud([n.resource for n in wallNodes],resolution=resolution)
    
    #remove all far away points
    dists = pcd.compute_point_cloud_distance(identityPcd)
    dists = np.asarray(dists)
    ind = np.where(dists <0.1)[0]
    inlier_pcd = pcd.select_by_index(ind)    

    #compute nearest neighbors
    indices,distances=gmu.compute_nearest_neighbors(np.asarray(inlier_pcd.points),np.asarray(identityPcd.points))    
    indices=indices.flatten()

    #compute the object and class scalars based on threshold distance
    labels = identityArray[indices[np.where(distances <= 0.1)[0]]]
    for u in np.unique( labels):
        object_pcd=inlier_pcd.select_by_index(np.where(labels==u)[0])
        wallNodes[u].pcd=object_pcd


processing 08_ShortOffice_01_F2_small_pred...


AttributeError: 'NoneType' object has no attribute 'flatten'

In [62]:
joined_mesh=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in wallNodes ])
joined_pcd=gmu.join_geometries([n.pcd for n in wallNodes ])

lines=gmu.mesh_get_lineset(joined_mesh)



o3d.visualization.draw_geometries([lines,joined_pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [8]:
files=[f for f in ut.get_list_of_files(input_folder) if f.endswith('.laz')]

#iterate over files
for f in files[:1]: 
    
    #read pcd
    print(f)
    las=laspy.read(f)
    
    #create nodes
    
    pcdNode=PointCloudNode(resource=gmu.las_to_pcd(las),
                                    name='_'.join(f.split('/')[-1].split('.')[0].split('_')[0:-1]),
                                    color=ut.random_color()) 
    
      
    class_scalar,object_scalar=t0.process_point_cloud(pcdNode,
                                                      [n for n in objectNodes if n.derivedFrom==pcdNode.name],
                                                      distance_threshold=distance_threshold,
                                                      resolution=resolution)
    
    gmu.las_add_extra_dimensions(las,(class_scalar,object_scalar),['classes','objects'],['uint8','uint16'])
    
    # las.write(output_folder/f'{pcdNode.name}_pcd.laz')
    # print(f'{pcdNode.name}_pcd.laz')
    # print(f'Elapsed time: {time.time() - start_time}')

c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/05_MedOffice_01_F2_s0p01m.laz
05_MedOffice_01_F2_pcd.laz
Elapsed time: 201.8166539669037
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/06_MedOffice_02_B1_s0p01m.laz
06_MedOffice_02_B1_pcd.laz
Elapsed time: 1159.0312247276306
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/06_MedOffice_02_F1_s0p01m.laz
06_MedOffice_02_F1_pcd.laz
Elapsed time: 2279.401542901993
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/06_MedOffice_02_F2_s0p01m.laz
06_MedOffice_02_F2_pcd.laz
Elapsed time: 3060.8463344573975
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/06_MedOffice_02_F3_s0p01m.laz
06_MedOffice_02_F3_pcd.laz
Elapsed time: 3716.6023156642914
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BI

## ADJUSTING FOR DOORS

In [24]:

input_folder2=path / 't1'/'input'

files=[f for f in ut.get_list_of_files(input_folder) if f.endswith('.laz')]
f=files[0]
doorNodes=[n for n in objectNodes if n.derivedFrom in f and n.class_id==4]
print(len(doorNodes))
print(doorNodes[0].class_id,doorNodes[0].object_id)

58


AttributeError: 'MeshNode' object has no attribute 'object_id'

In [27]:

input_folder2=path / 't1'/'input'

files=[f for f in ut.get_list_of_files(input_folder2) if f.endswith('.laz')]

#iterate over files
for f in files: 
    
    #read pcd
    print(f)
    las=laspy.read(f)
    
    #get door nodes
    doorNodes=[n for n in objectNodes if n.derivedFrom in f and n.class_id==4]
    print(len(doorNodes))
    
    points=o3d.utility.Vector3dVector(las.xyz)
    for i,d in enumerate(doorNodes):
        box=d.resource.get_oriented_bounding_box()
        box=gmu.expand_box(box,0.05,0.05,0.05)
        indices=box.get_point_indices_within_bounding_box(points)
        las['classes'][indices]=d.class_id
        las['objects'][indices]=i+400
        

    name=ut.get_filename(f)
    las.write(input_folder2/f'{name}1.laz')
    print(f'{f}1.laz')
    print(f'Elapsed time: {time.time() - start_time}')

c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/input/05_MedOffice_01_F2_small.laz
58
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/input/05_MedOffice_01_F2_small.laz1.laz
Elapsed time: 1572.0031366348267
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/input/05_MedOffice_01_F2_small1.laz
58
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/input/05_MedOffice_01_F2_small1.laz1.laz
Elapsed time: 1591.1883232593536
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/input/19_MedOffice_07_F4_small.laz
20
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/input/19_MedOffice_07_F4_small.laz1.laz
Elapsed time: 1615.2180304527283
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/input/32_ShortOffice_05_F1_small.laz
27
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR

In [19]:
joined_geometries=gmu.join_geometries([d.resource for d in doorNodes if d.derivedFrom in f])
# lines=gmu.join_geometries([n.line.paint_uniform_color(n.color) for n in objectNodes if n.derivedFrom==pcdNode.name 
#                            if getattr(n,'line',None) is not None])

o3d.visualization.draw_geometries([joined_geometries])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [10]:
# o=0
# walls=gmu.join_geometries([n.resource.paint_uniform_color([1,0,0]) for n in objectNodes if n.derivedFrom==pcdNodes[o].name
#                            if n.class_name in ['walls']])
# columns=gmu.join_geometries([n.resource.paint_uniform_color([0,1,0]) for n in objectNodes if n.derivedFrom==pcdNodes[o].name
#                              if n.class_name in ['columns']])
# doors=gmu.join_geometries([n.resource.paint_uniform_color([0,0,1]) for n in objectNodes if n.derivedFrom==pcdNodes[o].name
#                             if n.class_name in ['doors']])
# joined_geometries=gmu.join_geometries([walls,columns,doors])
# print(joined_geometries)

# lines=gmu.join_geometries([n.line.paint_uniform_color(n.color) for n in objectNodes if n.derivedFrom==pcdNodes[o].name 
#                            if getattr(n,'line',None) is not None])

# o3d.visualization.draw_geometries([pcdNodes[o].resource] + [joined_geometries,lines])

## PROCESSING

In [11]:
# objectNodes= [n for n in objectNodes if n.derivedFrom==pcdNode.name]

In [12]:
# #create an identity point cloud of all the objectNodes
# identityPcd,objectArray=gmu.create_identity_point_cloud([n.resource for n in objectNodes if n.derivedFrom==pcdNode.name],resolution=0.05)
# classArray=np.array([int(n.class_id) for n in objectNodes if n.derivedFrom==pcdNode.name])[objectArray.astype(int)]
# print(len(classArray),len(objectArray))

In [13]:
# #compute distances
# distances_total=pcdNodes[o].resource.compute_point_cloud_distance(identityPcd)
# distances_total=np.asarray(distances_total)

In [14]:
# object_scalar = np.full(len(pcdNode.resource.points), 0, dtype=np.uint8)
# class_scalar = np.full(len(pcdNode.resource.points), 255, dtype=np.uint8)

In [15]:
# indices,distances=gmu.compute_nearest_neighbors(np.asarray(pcdNode.resource.points),np.asarray(identityPcd.points))

In [16]:
# indices=indices.flatten()
# threshold_indices = np.where(distances <= distance_threshold)[0]
# object_scalar[threshold_indices] = objectArray[indices[threshold_indices]]
# class_scalar[threshold_indices] = classArray[indices[threshold_indices]]

In [17]:
# #remap objectArray
# names=np.array([int(n.name) for n in objectNodes])
# object_scalar=names[object_scalar]

In [18]:
# #temp
# temp_pcd=pcdNode.resource.select_by_index(np.where(distances<distance_threshold)[0])
# o3d.visualization.draw_geometries([temp_pcd] + [joined_geometries])

## OUTPUTS

### export las file

In [19]:
# gmu.las_add_extra_dimensions(las,(class_scalar,object_scalar),['classes','objects'],['uint8','uint8'])
# print(list(las.point_format.dimension_names))

In [20]:
# las.write(output_folder/f'{pcdNode.name}_pcd.laz')
# print(f'{pcdNode.name}_pcd.laz')

### save graph

In [21]:
for name in np.unique(np.array([n.derivedFrom for n in objectNodes])):
    nodes=[n for n in objectNodes if n.derivedFrom==name]
    tl.nodes_to_graph(nodes,
                    graphPath=str(output_folder/f'{name}_graph.ttl'),
                    save=True)

save objectNodes geometries

In [22]:
for name in np.unique(np.array([n.derivedFrom for n in objectNodes])):
    nodes=[n for n in objectNodes if n.derivedFrom==name]
    t0.write_obj_with_submeshes(output_folder/f'{name}_objects.obj', [n.resource for n in nodes], [n.name for n in nodes])
    print(f'{name} written!')

05_MedOffice_01_F2 written!
06_MedOffice_02_B1 written!
06_MedOffice_02_F1 written!
06_MedOffice_02_F2 written!
06_MedOffice_02_F3 written!
07_MedOffice_03_F3 written!
07_MedOffice_03_F4 written!
07_MedOffice_03_F5 written!
16_Facility_01_F1 written!
19_MedOffice_07_F4 written!
32_ShortOffice_05_F1 written!
32_ShortOffice_05_F2 written!
32_ShortOffice_05_F3 written!
33_SmallBuilding_03_F1 written!
35_Lab_02_F1 written!
35_Lab_02_F2 written!


## CHECK PTH

In [5]:
pth_folder=Path(os.getcwd()).parents[2]/'data'/'t1'
print(pth_folder)

c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t1


In [14]:
pcdNodes = []
import torch
#reload pth file 
for f in utl.get_list_of_files(pth_folder,ext='.pth')[:1]:
    print(f)
    
    chunk_dict=torch.load(f)
    #create o3d point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(chunk_dict['coord'])
    pcd.colors = o3d.utility.Vector3dVector(chunk_dict['color'])
    pcd.normals = o3d.utility.Vector3dVector(chunk_dict['normal'])
    colors=gmu.array_to_colors(chunk_dict['semantic_gt'])
    pcd.colors =o3d.utility.Vector3dVector(colors)
    pcdNodes.append(PointCloudNode(resource=pcd))
    
    labels,counts=np.unique(chunk_dict['semantic_gt'],return_counts=True)
    print(labels,counts)

c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t1/01_ITC_01_01_1_1_0.pth
[0 1 2 3 5] [1875177  681177      42  218249  919675]


In [19]:
pcd_temp=pcd.select_by_index(np.where(chunk_dict['semantic_gt']==3)[0])

In [20]:
o3d.visualization.draw_geometries([pcd_temp])

In [35]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.random_color()) for n in pcdNodes if n.resource is not None])

o3d.visualization.draw_geometries([joined_pcd])

## CHECK DIVIDE BOX

In [21]:
path=Path(os.getcwd()).parents[2]/'data'
input_folder=path / 't0'/'test'
print( input_folder)

for f in ut.get_list_of_files(input_folder)[-2:-1]:
    print(f)
    las=laspy.read(f)
    pcd=gmu.las_to_pcd(las)
    # pcdNode=PointCloudNode(resource=gmu.las_to_pcd(las),
    #                                 name='_'.join(f.split('/')[-1].split('.')[0].split('_')[0:-1]),
    #                                 color=ut.random_color()) 
    
    # class_scalar,object_scalar=t0.process_point_cloud(pcdNode,
    #                                                   [n for n in objectNodes if n.derivedFrom==pcdNode.name],
    #                                                   distance_threshold=distance_threshold,
    #                                                   resolution=resolution)
    
    # gmu.las_add_extra_dimensions(las,(class_scalar,object_scalar),['classes','objects'],['uint8','uint16'])
    
    # las.write(output_folder/f'{pcdNode.name}_pcd.laz')
    # print(f'{pcdNode.name}_pcd.laz')
    # print(f'Elapsed time: {time.time() - start_time}')
 
#  #divide the point cloud into chunks per part [7,7,1] or size e.g. [10m,10m,100m]
    

c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t0\test
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/test/25_Parking_01_F2_small.laz


In [22]:
size=[12,12,100]
parts=None
sub_pcds=[]
if size or parts:
    box=o3d.geometry.AxisAlignedBoundingBox(min_bound=np.array([las.x.min()-5,las.y.min()-5,las.z.min()-5]),
                                    max_bound=np.array([las.x.max()+5,las.y.max()+5,las.z.max()+5]))

    #create open3d axis aligned bounding box of points
    boxes,names=gmu.divide_box_in_boxes(box,size=size) if size is not None else gmu.divide_box_in_boxes(box,parts=parts)
    
    # select indices per boxes
    pathLists=[]
    idxLists=[]
    for box,name in zip(boxes,names):
        pathLists.append(f'{name[0]}_{name[1]}_{name[2]}')
        idxLists.append(box.get_point_indices_within_bounding_box(pcd.points))
    #save the chunks
    for i,name in zip(idxLists,pathLists):
        #select points
        sub_pcd=pcd.select_by_index(i)
        sub_pcds.append(sub_pcd)

In [23]:
for b in boxes:
    b.color=[1,0,0]
joined_pcd=gmu.join_geometries([n.paint_uniform_color(ut.random_color()) for n in sub_pcds])
# joined_pcd=gmu.join_geometries([n for n in sub_pcds])

o3d.visualization.draw_geometries([pcd]+[joined_pcd]+boxes)

## FINISH

In [23]:
now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

print("runtime: --- %s seconds ---" % (time.time() - start_time))


Current Time = 00:46:39
runtime: --- 9340.315954208374 seconds ---
